In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import os

### Merging both the throttle and steer dataframes shows that there are no matching time frames for both datasets 

In [2]:
steer = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/vehicle-steering_report.csv')
throttle = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/vehicle-throttle_info_report.csv')

In [3]:
steer.shape

(11040, 17)

In [4]:
steer.dtypes

Time                        float64
header.seq                    int64
header.stamp.secs             int64
header.stamp.nsecs            int64
header.frame_id             float64
steering_wheel_angle        float64
steering_wheel_angle_cmd    float64
steering_wheel_torque       float64
speed                       float64
enabled                        bool
override                       bool
driver                         bool
fault_wdc                      bool
fault_bus1                     bool
fault_bus2                     bool
fault_calibration              bool
fault_connector                bool
dtype: object

In [5]:
steer.isnull().sum()

Time                            0
header.seq                      0
header.stamp.secs               0
header.stamp.nsecs              0
header.frame_id             11040
steering_wheel_angle            0
steering_wheel_angle_cmd        0
steering_wheel_torque           0
speed                           0
enabled                         0
override                        0
driver                          0
fault_wdc                       0
fault_bus1                      0
fault_bus2                      0
fault_calibration               0
fault_connector                 0
dtype: int64

In [6]:
throttle.shape

(22118, 8)

In [7]:
throttle.dtypes

Time                  float64
header.seq              int64
header.stamp.secs       int64
header.stamp.nsecs      int64
header.frame_id       float64
throttle_pc           float64
throttle_rate         float64
engine_rpm            float64
dtype: object

In [8]:
throttle.isnull().sum()

Time                      0
header.seq                0
header.stamp.secs         0
header.stamp.nsecs        0
header.frame_id       22118
throttle_pc               0
throttle_rate             0
engine_rpm                0
dtype: int64

In [9]:
throttle_steer = steer.merge(throttle,on = 'Time',how = 'left')
throttle_steer.head()

,Time,header.seq_x,header.stamp.secs_x,header.stamp.nsecs_x,header.frame_id_x,steering_wheel_angle,steering_wheel_angle_cmd,steering_wheel_torque,speed,enabled,...,fault_bus2,fault_calibration,fault_connector,header.seq_y,header.stamp.secs_y,header.stamp.nsecs_y,header.frame_id_y,throttle_pc,throttle_rate,engine_rpm
0,1.479424e+09,278688,1479424214,787219361,NaN,0.003491,0.0,0.1250,22.880556,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.479424e+09,278689,1479424214,807231859,NaN,0.003491,0.0,0.1250,22.886110,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.479424e+09,278690,1479424214,827300730,NaN,0.003491,0.0,0.1250,22.891666,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.479424e+09,278691,1479424214,847355643,NaN,0.003491,0.0,0.1875,22.897223,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.479424e+09,278692,1479424214,867355824,NaN,0.003491,0.0,0.0000,22.902779,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
throttle_steer.isnull().sum()

Time                            0
header.seq_x                    0
header.stamp.secs_x             0
header.stamp.nsecs_x            0
header.frame_id_x           11040
steering_wheel_angle            0
steering_wheel_angle_cmd        0
steering_wheel_torque           0
speed                           0
enabled                         0
override                        0
driver                          0
fault_wdc                       0
fault_bus1                      0
fault_bus2                      0
fault_calibration               0
fault_connector                 0
header.seq_y                11040
header.stamp.secs_y         11040
header.stamp.nsecs_y        11040
header.frame_id_y           11040
throttle_pc                 11040
throttle_rate               11040
engine_rpm                  11040
dtype: int64

## Check some of the time differences and see that it is not obvious how to merge on the time column

In [11]:
def attach_image(dataframe, images_location):
    dfs = []
    for frame, image_dir in zip(dataframe, images_location):

        # Attaching image files to the inputs
        image_base_dir = f'./Data/ROSBAG/Ch2_002/images/{image_dir}'
            
        # Define camera folders
        camera_folders = {
                "left": os.path.join(image_base_dir, "left"),
                "center": os.path.join(image_base_dir, "center"),
                "right": os.path.join(image_base_dir, "right"),
            }
            
        # Create new columns for each camera
        for camera, folder in camera_folders.items():
            column_name = f'{camera}_image_path'
            image_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
                
            # Add the image paths to the DataFrame
            frame[column_name] = pd.Series(image_files)

        dfs.append(frame)
        
    updated_dataframe = pd.concat(dfs, ignore_index=True)
    
    return updated_dataframe

In [12]:
steer = attach_image([steer],['HMB_1'])
throttle = attach_image([throttle],['HMB_1'])

In [13]:
steer = steer.drop(columns = ['header.seq','header.stamp.secs','header.stamp.nsecs','header.frame_id','steering_wheel_angle_cmd',
                              'enabled','override','driver','fault_wdc','fault_bus1','fault_bus2','fault_calibration','fault_connector'])
throttle = throttle.drop(columns = ['header.seq','header.stamp.secs','header.stamp.nsecs','header.frame_id',])

# speed is in m/s so change to km/h
steer['speed'] = steer['speed'] * 3.6

steer.to_csv('./cleaned_data/steer_data_HMB_1.csv', index = False)
throttle.to_csv('./cleaned_data/throttle_data_HMB_1.csv', index = False)


In [14]:
steer.head()

,Time,steering_wheel_angle,steering_wheel_torque,speed,left_image_path,center_image_path,right_image_path
0,1.479424e+09,0.003491,0.1250,82.370002,./Data/ROSBAG/Ch2_002/images/HMB_1\left\frame_...,./Data/ROSBAG/Ch2_002/images/HMB_1\center\fram...,./Data/ROSBAG/Ch2_002/images/HMB_1\right\frame...
1,1.479424e+09,0.003491,0.1250,82.389997,./Data/ROSBAG/Ch2_002/images/HMB_1\left\frame_...,./Data/ROSBAG/Ch2_002/images/HMB_1\center\fram...,./Data/ROSBAG/Ch2_002/images/HMB_1\right\frame...
2,1.479424e+09,0.003491,0.1250,82.409999,./Data/ROSBAG/Ch2_002/images/HMB_1\left\frame_...,./Data/ROSBAG/Ch2_002/images/HMB_1\center\fram...,./Data/ROSBAG/Ch2_002/images/HMB_1\right\frame...
3,1.479424e+09,0.003491,0.1875,82.430001,./Data/ROSBAG/Ch2_002/images/HMB_1\left\frame_...,./Data/ROSBAG/Ch2_002/images/HMB_1\center\fram...,./Data/ROSBAG/Ch2_002/images/HMB_1\right\frame...
4,1.479424e+09,0.003491,0.0000,82.450003,./Data/ROSBAG/Ch2_002/images/HMB_1\left\frame_...,./Data/ROSBAG/Ch2_002/images/HMB_1\center\fram...,./Data/ROSBAG/Ch2_002/images/HMB_1\right\frame...


In [15]:
steer['Time'].diff().describe()

count    11039.000000
mean         0.020034
std          0.000508
min          0.013540
25%          0.019989
50%          0.020033
75%          0.020078
max          0.026501
Name: Time, dtype: float64

In [16]:
throttle.head()

,Time,throttle_pc,throttle_rate,engine_rpm,left_image_path,center_image_path,right_image_path
0,1.479424e+09,0.418,0.0,2770.00,./Data/ROSBAG/Ch2_002/images/HMB_1\left\frame_...,./Data/ROSBAG/Ch2_002/images/HMB_1\center\fram...,./Data/ROSBAG/Ch2_002/images/HMB_1\right\frame...
1,1.479424e+09,0.418,0.0,2762.25,./Data/ROSBAG/Ch2_002/images/HMB_1\left\frame_...,./Data/ROSBAG/Ch2_002/images/HMB_1\center\fram...,./Data/ROSBAG/Ch2_002/images/HMB_1\right\frame...
2,1.479424e+09,0.418,0.0,2762.25,./Data/ROSBAG/Ch2_002/images/HMB_1\left\frame_...,./Data/ROSBAG/Ch2_002/images/HMB_1\center\fram...,./Data/ROSBAG/Ch2_002/images/HMB_1\right\frame...
3,1.479424e+09,0.418,0.0,2754.75,./Data/ROSBAG/Ch2_002/images/HMB_1\left\frame_...,./Data/ROSBAG/Ch2_002/images/HMB_1\center\fram...,./Data/ROSBAG/Ch2_002/images/HMB_1\right\frame...
4,1.479424e+09,0.418,0.0,2747.25,./Data/ROSBAG/Ch2_002/images/HMB_1\left\frame_...,./Data/ROSBAG/Ch2_002/images/HMB_1\center\fram...,./Data/ROSBAG/Ch2_002/images/HMB_1\right\frame...


In [17]:
throttle['Time'].diff().describe()

count    22117.000000
mean         0.010000
std          0.001973
min          0.001279
25%          0.008471
50%          0.010028
75%          0.011515
max          0.018616
Name: Time, dtype: float64

In [18]:
os.listdir('./Data/ROSBAG/Ch2_002/images/HMB_1/left/')

['.ipynb_checkpoints',
 'frame_1479424215877601572.png',
 'frame_1479424215927376704.png',
 'frame_1479424215977463338.png',
 'frame_1479424216027468320.png',
 'frame_1479424216077857057.png',
 'frame_1479424216127646122.png',
 'frame_1479424216176569089.png',
 'frame_1479424216227025263.png',
 'frame_1479424216276615983.png',
 'frame_1479424216326904145.png',
 'frame_1479424216376863356.png',
 'frame_1479424216426864732.png',
 'frame_1479424216476992614.png',
 'frame_1479424216528110611.png',
 'frame_1479424216577422665.png',
 'frame_1479424216626893541.png',
 'frame_1479424216676870686.png',
 'frame_1479424216727342546.png',
 'frame_1479424216777440999.png',
 'frame_1479424216827011822.png',
 'frame_1479424216877123171.png',
 'frame_1479424216927135266.png',
 'frame_1479424216977107650.png',
 'frame_1479424217027394761.png',
 'frame_1479424217077616319.png',
 'frame_1479424217127387945.png',
 'frame_1479424217177012226.png',
 'frame_1479424217227095540.png',
 'frame_14794242172770503

In [19]:
'frame_1479424215877601572.png'.split('_')[1].split('.')[0]

'1479424215877601572'

In [20]:
left_timestamps = [row.split('_')[1].split('.')[0] for row in os.listdir('./Data/ROSBAG/Ch2_002/images/HMB_1/left/')]

In [21]:
float('1479424215877601572'[:10] + '.' + '1479424215877601572'[10:])

1479424215.8776016

## Merge after attaching image data to the csv

In [22]:
image_loc = ['HMB_1','HMB_2','HMB_4','HMB_5','HMB_6']

### IMU data

In [23]:
imu_hmb_1 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/imu-data.csv')
imu_hmb_2 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_2/imu-data.csv')
imu_hmb_4 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_4/imu-data.csv')
imu_hmb_5 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_5/imu-data.csv')
imu_hmb_6 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_6/imu-data.csv')
imu_dfs = [imu_hmb_1,imu_hmb_2,imu_hmb_4,imu_hmb_5,imu_hmb_6]
imu_all = attach_image(imu_dfs,image_loc)
imu_all.columns

Index(['Time', 'header.seq', 'header.stamp.secs', 'header.stamp.nsecs',
       'header.frame_id', 'orientation.x', 'orientation.y', 'orientation.z',
       'orientation.w', 'orientation_covariance_0', 'orientation_covariance_1',
       'orientation_covariance_2', 'orientation_covariance_3',
       'orientation_covariance_4', 'orientation_covariance_5',
       'orientation_covariance_6', 'orientation_covariance_7',
       'orientation_covariance_8', 'angular_velocity.x', 'angular_velocity.y',
       'angular_velocity.z', 'angular_velocity_covariance_0',
       'angular_velocity_covariance_1', 'angular_velocity_covariance_2',
       'angular_velocity_covariance_3', 'angular_velocity_covariance_4',
       'angular_velocity_covariance_5', 'angular_velocity_covariance_6',
       'angular_velocity_covariance_7', 'angular_velocity_covariance_8',
       'linear_acceleration.x', 'linear_acceleration.y',
       'linear_acceleration.z', 'linear_acceleration_covariance_0',
       'linear_accelerat

In [24]:
imu_all.drop(columns =['Time','header.seq','header.stamp.secs','header.stamp.nsecs', 'header.frame_id'],inplace= True)
imu_all = imu_all.dropna()

In [25]:
imu_all.to_csv('./Data/cleaned_data/imu_all.csv', index = False)

### Brake data

In [26]:
brake_hmb_1 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/vehicle-brake_info_report.csv')
brake_hmb_2 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_2/vehicle-brake_info_report.csv')
brake_hmb_4 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_4/vehicle-brake_info_report.csv')
brake_hmb_5 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_5/vehicle-brake_info_report.csv')
brake_hmb_6 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_6/vehicle-brake_info_report.csv')
brake_dfs = [brake_hmb_1,brake_hmb_2,brake_hmb_4,brake_hmb_5,brake_hmb_6]
brake_all = attach_image(brake_dfs,image_loc)
brake_all.columns

Index(['Time', 'header.seq', 'header.stamp.secs', 'header.stamp.nsecs',
       'header.frame_id', 'brake_torque_request', 'brake_torque_actual',
       'wheel_torque_actual', 'accel_over_ground', 'hsa.status', 'hsa.mode',
       'abs_active', 'abs_enabled', 'stab_active', 'stab_enabled',
       'trac_active', 'trac_enabled', 'parking_brake.status', 'stationary',
       'left_image_path', 'center_image_path', 'right_image_path'],
      dtype='object')

In [27]:
brake_all = brake_all.rename(columns = {'hsa.status': 'hill_start_assist_system_status',
                                       'hsa.mode':'hill_start_assist_mode',
                                       'parking_brake.status':'parking_brake_enabled'})

brake_all['stationary'] = brake_all['stationary'].astype(int)
brake_all['abs_active'] = brake_all['abs_active'].astype(int)
brake_all['abs_enabled'] = brake_all['abs_enabled'].astype(int)
brake_all['stab_active'] = brake_all['stab_active'].astype(int)
brake_all['stab_enabled'] = brake_all['stab_enabled'].astype(int)
brake_all['trac_active'] = brake_all['trac_active'].astype(int)
brake_all['trac_enabled'] = brake_all['trac_enabled'].astype(int)

brake_all.drop(columns = ['Time','header.seq', 'header.stamp.secs', 'header.stamp.nsecs',
                            'header.frame_id', 'brake_torque_request'], inplace = True)
brake_all = brake_all.dropna()

In [28]:
brake_all.to_csv('./Data/cleaned_data/brake_all.csv', index = False)

### Steering Data

In [29]:
steer_hmb_1 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/vehicle-steering_report.csv')
steer_hmb_2 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_2/vehicle-steering_report.csv')
steer_hmb_4 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_4/vehicle-steering_report.csv')
steer_hmb_5 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_5/vehicle-steering_report.csv')
steer_hmb_6 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_6/vehicle-steering_report.csv')
steer_dfs = [steer_hmb_1,steer_hmb_2,steer_hmb_4,steer_hmb_5,steer_hmb_6]
steer_all = attach_image(steer_dfs,image_loc)
steer_all.columns

Index(['Time', 'header.seq', 'header.stamp.secs', 'header.stamp.nsecs',
       'header.frame_id', 'steering_wheel_angle', 'steering_wheel_angle_cmd',
       'steering_wheel_torque', 'speed', 'enabled', 'override', 'driver',
       'fault_wdc', 'fault_bus1', 'fault_bus2', 'fault_calibration',
       'fault_connector', 'left_image_path', 'center_image_path',
       'right_image_path'],
      dtype='object')

In [30]:
steer_all['enabled'] = steer_all['enabled'].astype(int)

steer_all = steer_all.rename(columns = {'enabled': 'steering_enabled'})

steer_all['speed'] = steer_all['speed'] * 3.6

steer_all.drop(columns = ['Time','header.seq','header.stamp.secs','header.stamp.nsecs','header.frame_id','steering_wheel_angle_cmd',
                          'override','driver','fault_wdc','fault_bus1','fault_bus2','fault_calibration','fault_connector'],inplace = True)
steer_all = steer_all.dropna()

In [31]:
steer_all.to_csv('./Data/cleaned_data/steer_all.csv', index = False)

### Throttle Data

In [32]:
throttle_hmb_1 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/vehicle-throttle_info_report.csv')
throttle_hmb_2 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_2/vehicle-throttle_info_report.csv')
throttle_hmb_4 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_4/vehicle-throttle_info_report.csv')
throttle_hmb_5 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_5/vehicle-throttle_info_report.csv')
throttle_hmb_6 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_6/vehicle-throttle_info_report.csv')
throttle_dfs = [throttle_hmb_1,throttle_hmb_2,throttle_hmb_4,throttle_hmb_5,throttle_hmb_6]
throttle_all = attach_image(throttle_dfs,image_loc)
throttle_all.columns

Index(['Time', 'header.seq', 'header.stamp.secs', 'header.stamp.nsecs',
       'header.frame_id', 'throttle_pc', 'throttle_rate', 'engine_rpm',
       'left_image_path', 'center_image_path', 'right_image_path'],
      dtype='object')

In [33]:
throttle_all.drop(columns = ['Time','header.seq','header.stamp.secs','header.stamp.nsecs','header.frame_id'], inplace = True)
throttle_all = throttle_all.dropna()

In [34]:
throttle_all.to_csv('./Data/cleaned_data/throttle_all.csv', index = False)

In [35]:
throttle_all['engine_rpm'].mean()

1561.7455302963028

### Throttle Input Data

In [36]:
throttle_input_hmb_1 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/vehicle-throttle_report.csv')
throttle_input_hmb_2 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_2/vehicle-throttle_report.csv')
throttle_input_hmb_4 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_4/vehicle-throttle_report.csv')
throttle_input_hmb_5 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_5/vehicle-throttle_report.csv')
throttle_input_hmb_6 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_6/vehicle-throttle_report.csv')
throttle_input_dfs = [throttle_input_hmb_1,throttle_input_hmb_2,throttle_input_hmb_4,throttle_input_hmb_5,throttle_input_hmb_6]
throttle_input_all = attach_image(throttle_input_dfs,image_loc)
throttle_input_all.columns

Index(['Time', 'header.seq', 'header.stamp.secs', 'header.stamp.nsecs',
       'header.frame_id', 'pedal_input', 'pedal_cmd', 'pedal_output',
       'enabled', 'override', 'driver', 'watchdog_counter.source', 'fault_wdc',
       'fault_ch1', 'fault_ch2', 'fault_connector', 'left_image_path',
       'center_image_path', 'right_image_path'],
      dtype='object')

In [37]:
throttle_input_all['enabled'] = throttle_input_all['enabled'].astype(int)

throttle_input_all = throttle_input_all.rename(columns = {'enabled':'throttle_enabled',
                                                         'pedal_input':'throttle_pedal_input',
                                                         'pedal_cmd':'throttle_pedal_cmd',
                                                         'pedal_output':'throttle_pedal_output'})

throttle_input_all.drop(columns =['Time','header.seq','header.stamp.secs','header.stamp.nsecs','header.frame_id','override',
                                  'driver','watchdog_counter.source','fault_wdc','fault_ch1','fault_ch2','fault_connector'],inplace = True)
throttle_input_all = throttle_input_all.dropna()

In [38]:
throttle_input_all.to_csv('./Data/cleaned_data/throttle_input_all.csv', index = False)

### Gear State Data

In [39]:
gear_hmb_1 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/vehicle-gear_report.csv')
gear_hmb_2 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_2/vehicle-gear_report.csv')
gear_hmb_4 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_4/vehicle-gear_report.csv')
gear_hmb_5 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_5/vehicle-gear_report.csv')
gear_hmb_6 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_6/vehicle-gear_report.csv')
gear_dfs = [gear_hmb_1,gear_hmb_2,gear_hmb_4,gear_hmb_5,gear_hmb_6]
gear_all = attach_image(gear_dfs,image_loc)
gear_all.columns

Index(['Time', 'header.seq', 'header.stamp.secs', 'header.stamp.nsecs',
       'header.frame_id', 'state.gear', 'cmd.gear', 'override', 'fault_bus',
       'left_image_path', 'center_image_path', 'right_image_path'],
      dtype='object')

In [41]:
gear_all.drop(columns =['Time','header.seq','header.stamp.secs','header.stamp.nsecs','header.frame_id',
                        'override','fault_bus'],inplace = True)
gear_all = gear_all.rename(columns = {'state.gear':'gear_state',
                                     'cmd.gear':'gear_cmd'})

gear_all = gear_all.dropna()

In [42]:
gear_all.to_csv('./Data/cleaned_data/gear_all.csv', index = False)

### Brake Inputs and State Data

In [43]:
brake_inputs_hmb_1 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/vehicle-brake_report.csv')
brake_inputs_hmb_2 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_2/vehicle-brake_report.csv')
brake_inputs_hmb_4 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_4/vehicle-brake_report.csv')
brake_inputs_hmb_5 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_5/vehicle-brake_report.csv')
brake_inputs_hmb_6 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_6/vehicle-brake_report.csv')
brake_inputs_dfs = [brake_inputs_hmb_1,brake_inputs_hmb_2,brake_inputs_hmb_4,brake_inputs_hmb_5,brake_inputs_hmb_6]
brake_inputs_all = attach_image(brake_inputs_dfs,image_loc)
brake_inputs_all.columns

Index(['Time', 'header.seq', 'header.stamp.secs', 'header.stamp.nsecs',
       'header.frame_id', 'pedal_input', 'pedal_cmd', 'pedal_output',
       'torque_input', 'torque_cmd', 'torque_output', 'boo_input', 'boo_cmd',
       'boo_output', 'enabled', 'override', 'driver',
       'watchdog_counter.source', 'watchdog_braking', 'fault_wdc', 'fault_ch1',
       'fault_ch2', 'fault_boo', 'fault_connector', 'left_image_path',
       'center_image_path', 'right_image_path'],
      dtype='object')

In [44]:
brake_inputs_all = brake_inputs_all.rename(columns = {'boo_input': 'brake_lights_input',
                                                     'boo_output':'brake_lights_output',
                                                     'pedal_input':'brake_pedal_input',
                                                     'pedal_output':'brake_pedal_output',
                                                     'enabled':'brake_enabled',
                                                     'pedal_cmd':'brake_pedal_cmd'})

brake_inputs_all['brake_lights_input'] = brake_inputs_all['brake_lights_input'].astype(int)
brake_inputs_all['brake_lights_output'] = brake_inputs_all['brake_lights_output'].astype(int)
brake_inputs_all['brake_enabled'] = brake_inputs_all['brake_enabled'].astype(int)

brake_inputs_all.drop(columns = ['Time','header.seq','header.stamp.secs','header.stamp.nsecs','header.frame_id',
                        'override', 'driver', 'watchdog_counter.source','watchdog_braking','fault_wdc','fault_ch1','fault_ch2',
                                 'fault_boo','fault_connector','boo_cmd'],inplace = True)
brake_inputs_all=brake_inputs_all.dropna()

In [45]:
brake_inputs_all.to_csv('./Data/cleaned_data/brake_inputs_all.csv', index = False)

### Wheel Speed Data

In [46]:
wheel_speed_hmb_1 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/vehicle-wheel_speed_report.csv')
wheel_speed_hmb_2 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_2/vehicle-wheel_speed_report.csv')
wheel_speed_hmb_4 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_4/vehicle-wheel_speed_report.csv')
wheel_speed_hmb_5 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_5/vehicle-wheel_speed_report.csv')
wheel_speed_hmb_6 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_6/vehicle-wheel_speed_report.csv')
wheel_speed_dfs = [wheel_speed_hmb_1,wheel_speed_hmb_2,wheel_speed_hmb_4,wheel_speed_hmb_5,wheel_speed_hmb_6]
wheel_speed_all = attach_image(wheel_speed_dfs,image_loc)
wheel_speed_all.columns

Index(['Time', 'header.seq', 'header.stamp.secs', 'header.stamp.nsecs',
       'header.frame_id', 'front_left', 'front_right', 'rear_left',
       'rear_right', 'left_image_path', 'center_image_path',
       'right_image_path'],
      dtype='object')

In [47]:
wheel_speed_all = wheel_speed_all.rename(columns = {'front_left': 'front_left_wheel',
                                                     'front_right':'front_right_wheel',
                                                     'rear_left':'rear_left_wheel',
                                                     'rear_right':'rear_right_wheel'})
wheel_speed_all.drop(columns =['Time','header.seq','header.stamp.secs','header.stamp.nsecs','header.frame_id'],inplace = True)
wheel_speed_all = wheel_speed_all.dropna()

In [48]:
wheel_speed_all.to_csv('./Data/cleaned_data/wheel_speed_all.csv', index = False)

### Confirm dataframes can be merged on the images. We see that some nulls appear so drop them in function that merges all dataframes

In [49]:
merge = pd.merge(imu_all,steer_all, on = ['left_image_path', 'center_image_path', 'right_image_path'], how ='outer')
merge.isnull().sum()

orientation.x                       0
orientation.y                       0
orientation.z                       0
orientation.w                       0
orientation_covariance_0            0
orientation_covariance_1            0
orientation_covariance_2            0
orientation_covariance_3            0
orientation_covariance_4            0
orientation_covariance_5            0
orientation_covariance_6            0
orientation_covariance_7            0
orientation_covariance_8            0
angular_velocity.x                  0
angular_velocity.y                  0
angular_velocity.z                  0
angular_velocity_covariance_0       0
angular_velocity_covariance_1       0
angular_velocity_covariance_2       0
angular_velocity_covariance_3       0
angular_velocity_covariance_4       0
angular_velocity_covariance_5       0
angular_velocity_covariance_6       0
angular_velocity_covariance_7       0
angular_velocity_covariance_8       0
linear_acceleration.x               0
linear_accel

In [50]:
merge = pd.merge(merge,brake_all, on = ['left_image_path', 'center_image_path', 'right_image_path'], how ='outer')
merge.isnull().sum()

orientation.x                       0
orientation.y                       0
orientation.z                       0
orientation.w                       0
orientation_covariance_0            0
orientation_covariance_1            0
orientation_covariance_2            0
orientation_covariance_3            0
orientation_covariance_4            0
orientation_covariance_5            0
orientation_covariance_6            0
orientation_covariance_7            0
orientation_covariance_8            0
angular_velocity.x                  0
angular_velocity.y                  0
angular_velocity.z                  0
angular_velocity_covariance_0       0
angular_velocity_covariance_1       0
angular_velocity_covariance_2       0
angular_velocity_covariance_3       0
angular_velocity_covariance_4       0
angular_velocity_covariance_5       0
angular_velocity_covariance_6       0
angular_velocity_covariance_7       0
angular_velocity_covariance_8       0
linear_acceleration.x               0
linear_accel

In [51]:
merge = pd.merge(merge,throttle_all, on = ['left_image_path', 'center_image_path', 'right_image_path'], how ='outer')
merge.isnull().sum()

orientation.x                       0
orientation.y                       0
orientation.z                       0
orientation.w                       0
orientation_covariance_0            0
orientation_covariance_1            0
orientation_covariance_2            0
orientation_covariance_3            0
orientation_covariance_4            0
orientation_covariance_5            0
orientation_covariance_6            0
orientation_covariance_7            0
orientation_covariance_8            0
angular_velocity.x                  0
angular_velocity.y                  0
angular_velocity.z                  0
angular_velocity_covariance_0       0
angular_velocity_covariance_1       0
angular_velocity_covariance_2       0
angular_velocity_covariance_3       0
angular_velocity_covariance_4       0
angular_velocity_covariance_5       0
angular_velocity_covariance_6       0
angular_velocity_covariance_7       0
angular_velocity_covariance_8       0
linear_acceleration.x               0
linear_accel

In [52]:
merge = pd.merge(merge,gear_all, on = ['left_image_path', 'center_image_path', 'right_image_path'], how ='outer')
merge.columns

Index(['orientation.x', 'orientation.y', 'orientation.z', 'orientation.w',
       'orientation_covariance_0', 'orientation_covariance_1',
       'orientation_covariance_2', 'orientation_covariance_3',
       'orientation_covariance_4', 'orientation_covariance_5',
       'orientation_covariance_6', 'orientation_covariance_7',
       'orientation_covariance_8', 'angular_velocity.x', 'angular_velocity.y',
       'angular_velocity.z', 'angular_velocity_covariance_0',
       'angular_velocity_covariance_1', 'angular_velocity_covariance_2',
       'angular_velocity_covariance_3', 'angular_velocity_covariance_4',
       'angular_velocity_covariance_5', 'angular_velocity_covariance_6',
       'angular_velocity_covariance_7', 'angular_velocity_covariance_8',
       'linear_acceleration.x', 'linear_acceleration.y',
       'linear_acceleration.z', 'linear_acceleration_covariance_0',
       'linear_acceleration_covariance_1', 'linear_acceleration_covariance_2',
       'linear_acceleration_covarian

In [53]:
merge.isnull().sum()

orientation.x               0
orientation.y               0
orientation.z               0
orientation.w               0
orientation_covariance_0    0
                           ..
throttle_pc                 0
throttle_rate               0
engine_rpm                  0
gear_state                  1
gear_cmd                    1
Length: 62, dtype: int64

In [54]:
merge.dropna(inplace = True)

In [55]:
def merge_dataframes_on_columns(dataframes, merge_columns=['left_image_path', 'center_image_path', 'right_image_path']):
    
    # Start with the first DataFrame
    merged_df = dataframes[0]
    
    # Merge the rest of the DataFrames
    for df in dataframes[1:]:
        merged_df = pd.merge(merged_df, df, on=merge_columns, how='outer')
        merged_df.dropna(inplace = True)
    
    return merged_df
model_inputs = merge_dataframes_on_columns([imu_all, steer_all, throttle_all, brake_all, throttle_input_all, 
                                            wheel_speed_all, brake_inputs_all, gear_all])

model_inputs.to_csv('./Data/cleaned_data/model_inputs.csv', index = False)

In [56]:
model_inputs.columns

Index(['orientation.x', 'orientation.y', 'orientation.z', 'orientation.w',
       'orientation_covariance_0', 'orientation_covariance_1',
       'orientation_covariance_2', 'orientation_covariance_3',
       'orientation_covariance_4', 'orientation_covariance_5',
       'orientation_covariance_6', 'orientation_covariance_7',
       'orientation_covariance_8', 'angular_velocity.x', 'angular_velocity.y',
       'angular_velocity.z', 'angular_velocity_covariance_0',
       'angular_velocity_covariance_1', 'angular_velocity_covariance_2',
       'angular_velocity_covariance_3', 'angular_velocity_covariance_4',
       'angular_velocity_covariance_5', 'angular_velocity_covariance_6',
       'angular_velocity_covariance_7', 'angular_velocity_covariance_8',
       'linear_acceleration.x', 'linear_acceleration.y',
       'linear_acceleration.z', 'linear_acceleration_covariance_0',
       'linear_acceleration_covariance_1', 'linear_acceleration_covariance_2',
       'linear_acceleration_covarian